# LATAM DE Challenge

## Consideraciones:

1. Dado el tamaño del dataset (no es grande) no se utilizará Spark debido a que no es necesario procesamiento distribuido.
2. Se asume que el json entregado es correcto y no tiene errores en alguno de sus campos. Por ejemplo, se asume que los emoji se encuentran correctamente armados desde el dataset
3. Para el desarrollo se emplea Google BigQuery. En este notebook no se encuentran las credenciales de servicio, tampoco en el código.
4. Este notebook almacena los resultados de la ejecución de las funciones en BigQuery.
5. Se creo un archivo Python para realizar benchmark del job en BigQuery. Esto es el archivo src/gcp_benchmark.py
6. Considerando que toda la ejecución del job se realiza en BigQuery, no aporta realizar un benchmark de memoria y cpu de cada script. Puesto que estos se ejecutan de forma local y lo único que realizan es enviar la SQL query usando la API de Google BigQuery y recibir el resultado, no realizan ningun procesamiento. Por otro lado, el uso de memoria y CPU es constante en todos los casos puesto que el output entregado por BigQuery siempre son 10 registros de similar tamaño y tipo de datos, por lo que el uso de recursos computacionales es constante y no influye en la realización de este desafío. Finalmente, no se realizan mediciones locales (client Python), si no todo de GCP BigQuery (server GCP).

## Forma de trabajo

Se uso Gitflow para manejo de ramas: main - develop y luego de develop se crean ramas "feature" que son mergeadas a "develop" para luego realizar un "release" hacia main.

## Estructura del notebook:

Por cada caso (Qn) y su enfoque (memory o time) se presenta en orden:
1. Consulta SQL realizada.
2. Explicación.
3. Tradeoffs.
4. Mediciones y comparativas entre los enfoques.

In [18]:
## Configuracion del proyecto
# El dataset se encuentra cargado en una tabla en BigQuery
# El filepath corresponde a la ubicación en BigQuery
PROJECT_ID: str = "de-challenge-gm"
DATASET_ID: str = "tweets"
TABLE_NAME: str = "farmers-protest-tweets"

file_path = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}"
print(file_path)

de-challenge-gm.tweets.farmers-protest-tweets


# Q1: top 10 fechas donde hay más tweets

## Q1 BigQuery SQL Query: MEMORY

```python 
Q1_MEMORY_QUERY: str = """
-- CTE: contar tweets por fecha
WITH date_counts AS (
  SELECT
    -- Convertir fecha a Date
    DATE(date) AS tweet_date,
    -- Contar numero de tweets por fecha
    COUNT(*) AS tweet_count
  FROM
    `{file_path}`
  GROUP BY
    DATE(date)
),
-- CTE: seleccionar top10 fechas con mas tweets
top_10_dates AS (
  SELECT tweet_date, tweet_count
  FROM date_counts
  ORDER BY tweet_count DESC
  LIMIT 10
),
-- CTE: contar tweets por user y fecha dentro de top10
user_counts AS (
  SELECT
    DATE(date) AS tweet_date,
    user.username,
    -- Contar el numero de tweets por user en cada fecha
    COUNT(*) AS user_tweet_count
  FROM
    `{file_path}`
  WHERE DATE(date) IN (SELECT tweet_date FROM top_10_dates)
  GROUP BY
    DATE(date), user.username
)
-- Por cada fecha seleccionar la fecha y user con mas tweets
SELECT
  t.tweet_date,
  -- Seleccionar el user con mas tweets en cada fecha
  ARRAY_AGG(u.username ORDER BY u.user_tweet_count DESC LIMIT 1)[OFFSET(0)] AS top_user
FROM
  top_10_dates t
JOIN
  user_counts u
ON
  t.tweet_date = u.tweet_date
GROUP BY
  t.tweet_date, t.tweet_count
-- Ordenar resultado de forma descedente
ORDER BY
  t.tweet_count DESC
"""
```


In [19]:
# Ejecución q1_memory
import importlib
import q1_memory
importlib.reload(q1_memory)
result = q1_memory.q1_memory(file_path)
print(result)

2024-07-05 20:12:30,303 - INFO - Starting: q1_memory
2024-07-05 20:12:30,304 - INFO - Ejecutando query
2024-07-05 20:12:32,674 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 2.37 segundos
2024-07-05 20:12:32,675 - INFO - Se uso query cache?: False
2024-07-05 20:12:32,676 - INFO - Bigquery Job Detail:
2024-07-05 20:12:32,676 - INFO - Job ID: 285537c8-bd20-4c14-933c-7a7b7ba0bb86
2024-07-05 20:12:32,677 - INFO - Job Status: DONE
2024-07-05 20:12:32,678 - INFO - Tiempo inicio en máquina GCP: 2024-07-06 00:12:31.422000+00:00
2024-07-05 20:12:32,679 - INFO - Tiempo fin en máquina GCP: 2024-07-06 00:12:32.373000+00:00
2024-07-05 20:12:32,679 - INFO - Tiempo de ejecución total en servidor GCP: 0.951 segundos
2024-07-05 20:12:32,680 - INFO - Tiempo de ejecución total en cliente (python): 2.37 segundos
2024-07-05 20:12:32,680 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.42 segundos
2024-07-05 20:12:32,681 - INFO - Bytes pro

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Pasos q1_memory:
1. Cuenta tweets por fecha: crea tabla temporal date_count, esto temporalmente aumenta el uso de la memoria pero posteriormente se libera. Luego cuenta número de tweets por cada fehca y agrupa los resultados por "tweet_date"
2. Identifica top10 fechas con más tweets: crea tabla temporal "top_10_dates", en donde almacena la data de los tweets por fecha y count para luego limitarlo en 10 registros. Con esto es eficiente en memoria pero costoso en tiempo CPU.
3. Dentro de estas 10 fechas, cuenta el número de tweets por usuario en cada fecha.
4. Selecciona el usuario con mayor número de tweets para cada fecha. Se utiliza "ARRAY_AGG" el cuál se encuentra optimizado para agregaciones, en especifico, acá se agrega usernames ordenados por user_tweet_count de forma descendente.

Para la construcción de la consulta previamente mostrada (q1_memory) se utilizaron los siguientes criterios para su construcción, considerando que la priorización es el uso de memoria, es decir, minimizar el uso de memoria:

* Para reducir el consumo de memoria deberia ir filtrando (reduciendo) el espacio de búsqueda por cada step. De forma que cuando termina un step, se libera recursos y avanza con el nuevo conjunto. De esta forma el sistema operativo termianndo cada paso libera y no mantiene ocupada la memoria ram con la totalidad del dataset durante toda la ejecución del job. 
* Se utilizo Common Table Expressions (CTE) secuenciales para procesar los datos por etapas. Con esto sólo se mantiene los datos que se utilizan en memoria y por cada paso del query plan se liberan los recursos no utilizados. 
* La ventaja del enfoque anterior es que en efecto se usa menor recursos de memoria, porque cada step del query job sólo emplea en memoria los datos con los cuales trabajan, esto se evidencia en que cada Step S0X (01, 02, ... 0A) el número de registros leidos disminuye hasta ser constante. Sin embargo, esto es más lento, dado que es un procesamiento secuencial con múltiples etapas.
* Desventaja del enfoque es que se realiza un full scan de la tabla.

#### Análisis q1_memory

* **uso de memoria**: eficiente. Progresivamente reduce el conjunto de datos residente en memoria por lo que libera recursos en cada step.
* **tiempo de ejecución**: alto. Dado que existen múltiples etapas de procesamiento aumenta el tiempo de ejecución.
* **uso de CPU**: intermedio. Requiere realizar un full scan de la tabla y realizar una agregación. Mientras más grande la tabla mayor tiempo de CPU, aunque esto va a depender si se realizo algun index. En este caso no se realizaron indexes ni se guardaron resultados de consultas en materialized views.
* **uso de disco**: alto inicialmente. Comienza alto por la ingestion pero posteriormente por cada step del query plan se ve que la cantidad de registros leidos y escritos va disminuyendo. 


#### Posible mejora (q1_memory):
* Paralizar los steps, sin embargo esto conlleva a un aumento de costos puesto que es mayor cantidad de nodos a utilizar.
* Realizar un particionamiento o clustering por usuario. Actualmente la tabla cargada en BigQuery se encuentra particionada por la fecha (date).

## Q1 BigQuery SQL Query: TIME

```python 
Q1_TIME_QUERY: str = """
-- Funcion para seleccionar el usuario con mas tweets en una fecha
CREATE TEMP FUNCTION TopUserForDate(date_users ARRAY<STRUCT<username STRING, count INT64>>)
RETURNS STRING
LANGUAGE js AS '''
  return date_users.reduce((a, b) => a.count > b.count ? a : b).username;
''';

-- CTE: contar tweets por user y fecha
WITH date_user_counts AS (
  SELECT
    -- Convertir fecha a Date
    DATE(date) AS tweet_date,
    user.username,
    -- Contar el numero de tweets por user en cada fecha
    COUNT(*) AS tweet_count
  FROM
    `{file_path}`
  GROUP BY
    DATE(date), user.username
),
-- CTE: agregar totales de tweets por fecha
--      y seleccionar users con mas tweets
date_totals AS (
  SELECT
    tweet_date,
    -- Sumar numero total de tweets en cada fecha
    SUM(tweet_count) AS total_count,
    -- Crear array de users con sus count correspondiente de tweets
    ARRAY_AGG(STRUCT(username, tweet_count AS count) ORDER BY tweet_count DESC LIMIT 1) AS top_users
  FROM
    date_user_counts
  GROUP BY
    tweet_date
)
-- Por cada fecha seleccionar la fecha y user con mas tweets
SELECT
  tweet_date,
  -- Uso de la funcion para obtener el user con mas tweets en cada fecha
  TopUserForDate(top_users) AS top_user
FROM (
  SELECT *
  FROM date_totals
  ORDER BY total_count DESC
  LIMIT 10
)
-- Order total de tweets descendente
ORDER BY
  total_count DESC
"""
```

In [3]:
# Ejecución q1_time
import q1_time
import importlib
importlib.reload(q1_time)
result = q1_time.q1_time(file_path)
print(result)

2024-07-05 14:17:17,526 - INFO - Starting: q1_time
2024-07-05 14:17:17,526 - INFO - Ejecutando query
2024-07-05 14:17:19,151 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.62 segundos
2024-07-05 14:17:19,152 - INFO - Se uso query cache?: False
2024-07-05 14:17:19,153 - INFO - Bigquery Job Detail:
2024-07-05 14:17:19,153 - INFO - Job ID: 80b46508-6a65-4ee4-9ecb-63c551d06751
2024-07-05 14:17:19,154 - INFO - Job Status: DONE
2024-07-05 14:17:19,154 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 18:17:18.112000+00:00
2024-07-05 14:17:19,155 - INFO - Tiempo fin en máquina GCP: 2024-07-05 18:17:18.789000+00:00
2024-07-05 14:17:19,155 - INFO - Tiempo de ejecución total en servidor GCP: 0.677 segundos
2024-07-05 14:17:19,156 - INFO - Tiempo de ejecución total en cliente (python): 1.62 segundos
2024-07-05 14:17:19,157 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 0.95 segundos
2024-07-05 14:17:19,157 - INFO - Bytes proce

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Pasos q1_time:

A diferencia del enfoque anterior, acá se implemento una función en Javascript (UDF) para selecionar el usuario con más tweets en una fecha.
1. Crea una función para seleccionar el usuario con más tweets en una fecha. Esto se realizo mediante un UDF debido a que personalmente me acomoda más escribir ese tipo de consultas mediante una sintaxis de paradigma funcional la cuál me provee Javascript (es un reduce), por otro lado, reduce realizar consultas SQLs complejas.
2. Cuenta el número de tweets por usuario y fecha.
3. Suma los totales de tweets por fecha y selecciona los usuarios con más tweets.
4. Usa la función previamente definida y almacenada en memoria para obtener de forma funcional (salida solo depende de las entradas) al usuario con más tweets en cada una de las 10 fechas.

Comparando con el enfoque q1_memory, el uso de la UDF simplifica y acelera la selección del usuario con más tweets. Esto reduce la cantidad de CTEs a almacenar puesto que este recurso de CTEs no se encuentra optimizadas para minimizar el volumen en las primeras etapas del procesamiento.

Luego, la agrupación y sumas ocurren en una sola operación, reduciendo el número de steps en la query plan. Finalmente, limita y ordena los resultados antes de procesar más datos, reduciendo el uso de CPU en las siguientes consultas.

#### Análisis q1_time

* **uso de memoria**: eficiente. Al limitar los datos a 10 fechas en las primeras fases del query plan, limita el uso de datos residentes en memoria.
* **tiempo de ejecución**: bajo. Al usar la UDF se optimiza el tiempo de uso de la CPU, es reducido comparado con el enfoque previo. Esto también ayuda a evitar múltiples etapas de procesamiento.
* **uso de CPU**: intermedio. Si bien la query realiza un full scan de la tabla y multiples agregacias, el uso de CPU no se dispara debido a que todo se realiza en una sóla ejecución, por lo que el tiempo de uso CPU constante durante toda la operación.
* **uso de disco**: alto inicialmente. Comienza alto por la ingestion pero posteriormente por cada step del query plan se ve que la cantidad de registros leidos y escritos va disminuyendo. 


#### Posible mejora (q1_time):
* Paralizar los steps, sin embargo esto conlleva a un aumento de costos puesto que es mayor cantidad de nodos a utilizar.
* Realizar un particionamiento o clustering por usuario. Actualmente la tabla cargada en BigQuery se encuentra particionada por la fecha (date).
* Usar precomputacion o cache de agregaciones.
* Usar indices en date y user.username. No obstante, la primera query debe crear esto para que las siguientes n+1 consultas puedan observar este beneficio.
* Usar SQL nativo si se maneja las optimizaciones de BigQuery. En mi caso desconozco optimizaciones a nivel de SQL con BigQuery.

## Tabla Comparativa de Mediciones Q1

### Configuración del job Q1

| Parámetros                                      | q1_memory | q1_time |
|------------------------------------------------|-----------|---------|
| QUERY_CACHE                                    | False     | False   |
| QUERY_PRIORITY                                 | BATCH     | INTERACTIVE |

### Resultados mediciones

| Medicion                                      | q1_memory | q1_time |
|-----------------------------------------------|-----------|---------|
| Tiempo ejecución en cliente                   | 2.37 s    | 1.62 s  |
| Tiempo ejecución en GCP (server)              | 0.951 s   | 0.677 s |
| Delta tiempo (client-server)                  | 1.42 s    | 0.95 s  |
| Memoria usada por cliente python              | NA        | NA      |
| CPU usado por cliente python                  | NA        | NA      |
| Bytes Procesados en GCP (server)              | 2.45 MB   | 2.45 MB |
| Bytes Facturados en GCP (server)              | 10.49 MB  | 10.49 MB|
| Slot Machine tiempo usado (GCP CPU Time) (ms) | 22696 ms  | 1890 ms |
| BigQuery Total steps en QueryPlan             | 11        | 5       |

### Resultados Query Plan

#### Query Plan: q1_memory

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 13              | COMPLETE |
| S01   | Sort+                | 13           | 10              | COMPLETE |
| S02   | Sort                 | 10           | 10              | COMPLETE |
| S03   | Coalesce             | 10           | 10              | COMPLETE |
| S04   | Join+                | 117537       | 44159           | COMPLETE |
| S06   | Sort                 | 10           | 10              | COMPLETE |
| S07   | Coalesce             | 10           | 10              | COMPLETE |
| S08   | Join+                | 44169        | 10              | COMPLETE |
| S09   | Aggregate+           | 10           | 10              | COMPLETE |
| S0A   | Output               | 10           | 10              | COMPLETE |

#### Query Plan: q1_time

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 51646           | COMPLETE |
| S01   | Aggregate+           | 51646        | 13              | COMPLETE |
| S02   | Sort+                | 13           | 10              | COMPLETE |
| S03   | Sort+                | 10           | 10              | COMPLETE |
| S04   | Output               | 10           | 10              | COMPLETE |

### Comparación Query Plan enfoque memory vs time

| Step | q1_memory (Q1_MEMORY_QUERY)                                   | q1_time (Q1_TIME_QUERY)                                    |
|------|---------------------------------------------------------------|-----------------------------------------------------------|
| S00  | **Input**: Records read: 117407, Records written: 13, Status: COMPLETE | **Input**: Records read: 117407, Records written: 51646, Status: COMPLETE |
| S01  | **Sort+**: Records read: 13, Records written: 10, Status: COMPLETE | **Aggregate+**: Records read: 51646, Records written: 13, Status: COMPLETE |
| S02  | **Sort**: Records read: 10, Records written: 10, Status: COMPLETE | **Sort+**: Records read: 13, Records written: 10, Status: COMPLETE |
| S03  | **Coalesce**: Records read: 10, Records written: 10, Status: COMPLETE | **Sort+**: Records read: 10, Records written: 10, Status: COMPLETE |
| S04  | **Join+**: Records read: 117537, Records written: 44159, Status: COMPLETE | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |
| S05  | **Aggregate**: Records read: 44159, Records written: 44159, Status: COMPLETE |                                                           |
| S07  | **Coalesce**: Records read: 44159, Records written: 44159, Status: COMPLETE |                                                           |
| S08  | **Join+**: Records read: 44169, Records written: 10, Status: COMPLETE |                                                           |
| S09  | **Aggregate+**: Records read: 10, Records written: 10, Status: COMPLETE |                                                           |
| S0A  | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |                                                           |



A partir de los resultados anteriores se evidencia de que q1_time tiene una mejora con respecto al tiempo de ejecución versus su contraparte q1_memory.
Esto se evidencia en los query plan de cada enfoque. Mientras que q1_memory tiene 10 pasos, q1_time tiene 4 (considerando output). En donde q1_memory va leyendo de a poco la data en cada paso, optimizando el uso de memoria durante la ejecución. Mientras que la version enfocada en el tiempo de ejecución se centra en obtener todo rápido (algo como un algoritmo voraz) realiza toda la ingesta y procesa todo sin depender de steps intermedios. La desventaja de esto es que el consumo de memoria y cpu es elevado. Sin embargo, la medición muestra que la máquina optimizo el uso de CPU y paso menos tiempo en CPU (dado la cantidad de steps).
Se destaca que sólo para este paso se uso una configuración de job queue Batch para verificar como funcionaba el queue en batch para el dataset actual. El resto de casos se ocupo en Interactive.

# Q2

* Para resolver Q2 se empleo rangos de regex para UNICODE. Acá hay una gran oportunidad de mejora, faltan rangos de unicode para emoticons por lo que es altamente probable que no capture todos los emoticons que existen dentro del dataset que se encuentra cargado en BigQuery.
* Sin embargo como se presenta a continuación, el código es mantenible en este apartado puesto que para agregar más rangos no es necesario cambiar la consulta (tanto de q2_time o q2_memory) si no que simplemente se agrega en la constante UNICODE_RANGES.

## Q2: SQL MEMORY

```python 
# Unicode ranges
EMOTICONS = r"\x{1F600}-\x{1F64F}"
MISC_SYMBOLS_PICTOGRAPHS = r"\x{1F300}-\x{1F5FF}"
TRANSPORT_MAP_SYMBOLS = r"\x{1F680}-\x{1F6FF}"
FLAGS = r"\x{1F1E6}-\x{1F1FF}"
MISC_SYMBOLS = r"\x{2600}-\x{26FF}"
DINGBATS = r"\x{2700}-\x{27BF}"
SUPPLEMENTAL_SYMBOLS_PICTOGRAPHS = r"\x{1F900}-\x{1F9FF}"
SYMBOLS_PICTOGRAPHS_EXTENDED = r"\x{1FA70}-\x{1FAFF}"

UNICODE_RANGES = (
    f"[{EMOTICONS}"
    f"{MISC_SYMBOLS_PICTOGRAPHS}"
    f"{TRANSPORT_MAP_SYMBOLS}"
    f"{FLAGS}"
    f"{MISC_SYMBOLS}"
    f"{DINGBATS}"
    f"{SUPPLEMENTAL_SYMBOLS_PICTOGRAPHS}"
    f"{SYMBOLS_PICTOGRAPHS_EXTENDED}]"
)

Q2_MEMORY_QUERY: str = rf"""
-- Funcion: extraer emojis únicos de un string
CREATE TEMP FUNCTION ExtractEmoji(content STRING) AS (
  -- usar ARRAY_AGG con DISTINCT para eliminar duplicados de inmediato
  (SELECT ARRAY_AGG(DISTINCT char IGNORE NULLS)
   -- Separa content en char
   FROM UNNEST(SPLIT(content, '')) AS char
   -- Filtra caracteres que hacen match con rangos unicode
   WHERE REGEXP_CONTAINS(char, r'{UNICODE_RANGES}'))
);

-- Main Query
WITH emoji_counts AS (
  -- Extrae y cuenta emojis en una pasada
  SELECT
    emoji,
    COUNT(*) as count
  FROM
     `{{file_path}}`,
    -- Usa la función para extraer emojis y hace unnest de los resultados
    UNNEST(ExtractEmoji(content)) as emoji
  -- Group por emoji unico
  GROUP BY
    emoji
)
-- Select the top 10 emojis
SELECT emoji, count
FROM (
  SELECT
    emoji,
    count,
    -- Asigna un ranking cada emoji basado en su count y desc
    RANK() OVER (ORDER BY count DESC) as rank
  FROM emoji_counts
)
WHERE rank <= 10
ORDER BY count DESC
"""
```

In [15]:
# Ejecución q2_memory
import q2_memory
import importlib
importlib.reload(q2_memory)
result = q2_memory.q2_memory(file_path)
print(result)

2024-07-05 19:55:54,180 - INFO - Starting: q2_memory
2024-07-05 19:55:54,180 - INFO - Ejecutando query
2024-07-05 19:55:56,127 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.95 segundos
2024-07-05 19:55:56,129 - INFO - Se uso query cache?: False
2024-07-05 19:55:56,129 - INFO - Bigquery Job Detail:
2024-07-05 19:55:56,130 - INFO - Job ID: 2314a991-5332-4252-ac8d-a5c8b0cf9a7a
2024-07-05 19:55:56,131 - INFO - Job Status: DONE
2024-07-05 19:55:56,131 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 23:55:54.612000+00:00
2024-07-05 19:55:56,132 - INFO - Tiempo fin en máquina GCP: 2024-07-05 23:55:55.682000+00:00
2024-07-05 19:55:56,132 - INFO - Tiempo de ejecución total en servidor GCP: 1.07 segundos
2024-07-05 19:55:56,133 - INFO - Tiempo de ejecución total en cliente (python): 1.95 segundos
2024-07-05 19:55:56,134 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 0.88 segundos
2024-07-05 19:55:56,134 - INFO - Bytes proc

[('✊', 1724), ('❤', 1471), ('✌', 202), ('☮', 170), ('♂', 139), ('♀', 113), ('✍', 91), ('♥', 56), ('⚔', 48), ('✅', 44)]


#### Pasos q2_memory:

1. Se define función para extraer emojis únicos, esto es útil para los count.
2. Contar los Emojis:
  2.1 Define una Common Table Expression (CTE) llamada emoji_counts que:
    Utiliza la función ExtractEmoji definido en el paso anterior para extraer emojis del contenido de los tweets.
    Descompone los arrays de emojis en elementos individuales usando UNNEST.
  2.2 Cuenta la frecuencia de aparición (COUNT) de cada emoji único.
  2.3 Agrupa los emojis únicos y sus conteos correspondientes.
3. Seleccionar los 10 Emojis Más Frecuentes:
    3.1 Subquery: Asignar un ranking a cada emoji basado en su conteo. Luego se ordena de forma descedente. Esto se hace con la función "RANK"
    3.2 Filtra los resultados para obtener sólo 10 emoji más frecuentes, es decir, ranking menor o igual a 10
    3.3 Ordenar emojis

#### Análisis q2_memory

* **uso de memoria**: bajo. "unnest" ayuda a trabajar los datos en pequeñas listas reduciendo la cantidad de datos en memoria por cada query step.
* **tiempo de ejecución**: intermedio. Las operaciones se realizan en una sola pasada/ejecución. Pero el tiempo puede variar considerando la cantidad de emojis por cada texto.
* **uso de CPU**: alto. Las operaciones de "unnest" y "regexp_contains" son computacionalmente caras en términos de uso de CPU.
* **uso de disco**: bajo. La query no requiere múltiples escrituras y lecturas de disco. Puesto que todo se realiza en memoria, es ecir, la extracción de los emojis y conteo se realiza en memoria, minimizando el uso de disco. La creación de tablas temporales y funciones BigQuery usa paginación para mantener la referencia "fácil acceso" en memoria principal por lo que la plataforma maneja estos aspectos de forma eficiente minimizando los accesos a disco.


#### Posible mejora (q2_memory):
* Clustering para agrupar campos de uso frecuente (ej: emojis frecuentes)
* Indices para acelerar las operaciones de regex y split. No obstante, disminuyes tiempo de CPU pero aumentas uso de storage debido al tamaño de los indices.

## Q2: SQL TIME

```python 
Q2_TIME_QUERY: str = rf"""
CREATE TEMP FUNCTION ExtractEmoji(content STRING) AS (
  (SELECT ARRAY_AGG(DISTINCT emoji IGNORE NULLS)
   FROM UNNEST(REGEXP_EXTRACT_ALL(content, r'{UNICODE_RANGES}')) AS emoji)
);

SELECT
  emoji,
  COUNT(*) AS count
FROM
  `{{file_path}}`,
  UNNEST(ExtractEmoji(content)) AS emoji
GROUP BY
  emoji
ORDER BY
  count DESC
LIMIT 10
"""
```

In [16]:
# Ejecución q2_time
import q2_time
import importlib
importlib.reload(q2_time)
result = q2_time.q2_time(file_path)
print(result)

2024-07-05 19:56:05,260 - INFO - Starting: q2_time
2024-07-05 19:56:05,261 - INFO - Ejecutando query
2024-07-05 19:56:06,831 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.57 segundos
2024-07-05 19:56:06,832 - INFO - Se uso query cache?: False
2024-07-05 19:56:06,832 - INFO - Bigquery Job Detail:
2024-07-05 19:56:06,833 - INFO - Job ID: feba3c6f-55e7-4472-81d5-1dab16de334c
2024-07-05 19:56:06,833 - INFO - Job Status: DONE
2024-07-05 19:56:06,834 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 23:56:05.682000+00:00
2024-07-05 19:56:06,834 - INFO - Tiempo fin en máquina GCP: 2024-07-05 23:56:06.092000+00:00
2024-07-05 19:56:06,835 - INFO - Tiempo de ejecución total en servidor GCP: 0.41 segundos
2024-07-05 19:56:06,835 - INFO - Tiempo de ejecución total en cliente (python): 1.57 segundos
2024-07-05 19:56:06,835 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.16 segundos
2024-07-05 19:56:06,836 - INFO - Bytes proces

[('✊', 1724), ('❤', 1471), ('✌', 202), ('☮', 170), ('♂', 139), ('♀', 113), ('✍', 91), ('♥', 56), ('⚔', 48), ('✅', 44)]


#### Pasos q2_time:

La principal diferencia con el enfoque anterior es que, ahora se define una función UDF igual que el q1_time para extraer emojis. Esta función "ExtractEmoji" utiliza "REGEXP_EXTRACT_ALL" que extrae todos los emoji de los contenidos de los tweets y agrega emoji únicos en un array usando "ARRAY_GG" con "DISTINCT"


#### Análisis q2_time

* **uso de memoria**: bajo. "unnest" ayuda a trabajar los datos en pequeñas listas reduciendo la cantidad de datos en memoria por cada query step.
* **tiempo de ejecución**: bajo en comparación con el enfoque anterior. Las operaciones computacionalmente caras se realizan en una sola pasada/ejecución.
* **uso de CPU**: alto. Las operaciones de "unnest" y "regexp_contains" son computacionalmente caras en términos de uso de CPU.
* **uso de disco**: bajo. Idem que enfoque anterior.


#### Posible mejora (q2_time):
* Agrupar emojis frecuentes para mejorar futuras consultas. Una especie de cache. Almacenar precomputaciones.
* Indices para acelerar instrucción de REGEX y split.

## Tabla Comparativa de Mediciones Q2

### Configuración del job Q2

| Parámetros     | q2_memory | q2_time |
|----------------|-----------|---------|
| QUERY_CACHE    | False     | False   |
| QUERY_PRIORITY | INTERACTIVE | INTERACTIVE |

### Resultados mediciones

| Medición                                      | q2_memory | q2_time |
|-----------------------------------------------|-----------|---------|
| Tiempo ejecución en cliente                   | 1.95 s    | 1.57 s  |
| Tiempo ejecución en GCP (server)              | 1.07 s    | 0.41 s  |
| Delta tiempo (client-server)                  | 0.88 s    | 1.16 s  |
| Memoria usada por cliente python              | NA        | NA      |
| CPU usado por cliente python                  | NA        | NA      |
| Bytes Procesados en GCP (server)              | 20.86 MB  | 20.86 MB|
| Bytes Facturados en GCP (server)              | 22.02 MB  | 22.02 MB|
| Slot Machine tiempo usado (GCP CPU Time) (ms) | 28137 ms  | 813 ms  |
| BigQuery Total steps en QueryPlan             | 10        | 3       |

### Resultados Query Plan

#### Query Plan: q2_memory

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 361             | COMPLETE |
| S01   | Aggregate            | 361          | 70              | COMPLETE |
| S02   | Aggregate+           | 70           | 1               | COMPLETE |
| S03   | Aggregate            | 1            | 1               | COMPLETE |
| S04   | Sort+                | 71           | 70              | COMPLETE |
| S05   | Aggregate            | 70           | 68              | COMPLETE |
| S06   | Aggregate+           | 68           | 32              | COMPLETE |
| S08   | Coalesce             | 32           | 32              | COMPLETE |
| S09   | Join+                | 1670         | 10              | COMPLETE |
| S0A   | Output               | 10           | 10              | COMPLETE |

#### Query Plan: q2_time

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 361             | COMPLETE |
| S01   | Sort+                | 361          | 10              | COMPLETE |
| S02   | Output               | 10           | 10              | COMPLETE |

### Comparación Query Plan enfoque memory vs time

| Step | q2_memory (Q2_MEMORY_QUERY)                                   | q2_time (Q2_TIME_QUERY)                                    |
|------|---------------------------------------------------------------|-----------------------------------------------------------|
| S00  | **Input**: Records read: 117407, Records written: 361, Status: COMPLETE | **Input**: Records read: 117407, Records written: 361, Status: COMPLETE |
| S01  | **Aggregate**: Records read: 361, Records written: 70, Status: COMPLETE | **Sort+**: Records read: 361, Records written: 10, Status: COMPLETE |
| S02  | **Aggregate+**: Records read: 70, Records written: 1, Status: COMPLETE | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |
| S03  | **Aggregate**: Records read: 1, Records written: 1, Status: COMPLETE |                                                           |
| S04  | **Sort+**: Records read: 71, Records written: 70, Status: COMPLETE |                                                           |
| S05  | **Aggregate**: Records read: 70, Records written: 68, Status: COMPLETE |                                                           |
| S06  | **Aggregate+**: Records read: 68, Records written: 32, Status: COMPLETE |                                                           |
| S08  | **Coalesce**: Records read: 32, Records written: 32, Status: COMPLETE |                                                           |
| S09  | **Join+**: Records read: 1670, Records written: 10, Status: COMPLETE |                                                           |
| S0A  | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |                                                           |

A partir de los resultados anteriores se evidencia de que q2_time tiene una mejora con respecto al tiempo de ejecución versus su contraparte q2_memory en casi un 50%.
Esto se evidencia en los query plan de cada enfoque. Mientras que q2_memory tiene 10 pasos, q2_time tiene 3 (considerando output). En donde q2_memory va leyendo de a poco la data en cada paso, optimizando el uso de memoria durante la ejecución. Mientras que la version enfocada en el tiempo de ejecución se centra en obtener todo rápido (algo como un algoritmo voraz) realiza toda la ingesta y procesa todo sin depender de steps intermedios. La desventaja de esto es que el consumo de memoria y cpu es elevado. Sin embargo, la medición muestra que la máquina optimizo el uso de CPU y paso menos tiempo en CPU (dado la cantidad de steps). Cabe destacar que el enfoque de q2_time es leer todo en el primer step, luego step sort y luego output. El overhead inicial de consulta es alto en termino de CPU, memoria y disco. Sin embargo, paso mucho menos tiempo en CPU por el tamaño del dataset. 
El dataset al ser pequeño logro realizar todo de forma rápida, logrando un uso minimo de tiempo CPU (813 ms) vs su contraparte memory (28137 ms). Sin embargo, tal como se ve en la comparación de query plan, si el dataset hubiera sido del orden de los GBs el overhead y uso de CPU se dispararía dada la cantidad de steps (pocos) en q2_time, tradeoffs, lo que gano en procesamiento no es escalable a un dataset más grande.

# Q3: Top 10 histórico de usuarios (username) más influyentes (más mencionados)

## Q3: SQL MEMORY

```python 
Q3_MEMORY_QUERY: str = """
WITH filtered_mentions AS (
  SELECT mentionedUser.username
  FROM `{file_path}`,
       UNNEST(mentionedUsers) AS mentionedUser
  WHERE mentionedUsers IS NOT NULL
    AND mentionedUser.username IS NOT NULL
)
SELECT username, COUNT(*) AS mention_count
FROM filtered_mentions
GROUP BY username
ORDER BY mention_count DESC
LIMIT 10
"""
```

In [7]:
# Ejecución q3_memory
import q3_memory
import importlib
importlib.reload(q3_memory)
result = q3_memory.q3_memory(file_path)
print(result)

2024-07-05 18:30:29,041 - INFO - Starting: q3_memory
2024-07-05 18:30:29,042 - INFO - Ejecutando query
2024-07-05 18:30:30,504 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.46 segundos
2024-07-05 18:30:30,505 - INFO - Se uso query cache?: False
2024-07-05 18:30:30,506 - INFO - Bigquery Job Detail:
2024-07-05 18:30:30,506 - INFO - Job ID: 6f194a9e-0755-4bc2-a0cb-4ef2d8b6da0c
2024-07-05 18:30:30,507 - INFO - Job Status: DONE
2024-07-05 18:30:30,507 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 22:30:29.521000+00:00
2024-07-05 18:30:30,508 - INFO - Tiempo fin en máquina GCP: 2024-07-05 22:30:29.943000+00:00
2024-07-05 18:30:30,508 - INFO - Tiempo de ejecución total en servidor GCP: 0.422 segundos
2024-07-05 18:30:30,509 - INFO - Tiempo de ejecución total en cliente (python): 1.46 segundos
2024-07-05 18:30:30,509 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.04 segundos
2024-07-05 18:30:30,510 - INFO - Bytes pro

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]


#### Pasos q3_memory:

1. Filtrar menciones de usuarios:
    1.1 Crea CTE: filtered_mentions:
        1.1.1 Selecciona los nombres de usuario mencionados (mentionedUser.username)
        1.1.2 Descompone los arrays de usuarios mencionados en elementos individuales usando UNNEST.
        1.1.3 Filtra los resultados para asegurarse de que "mentionedUsers" no sea nulo y que "mentionedUser.username" no sea nulo. Con esto ahorramos memoria en siguientes pasos del query plan.
2. Contar las menciones agrupando los nombres de usuario por el campo username.
3. Selecciona el top10 usuarios ordenando por mention_count descendente y limitando los resultados.


#### Análisis q3_memory

* **uso de memoria**: bajo. "unnest" ayuda a trabajar los datos en pequeñas listas reduciendo la cantidad de datos en memoria por cada query step. A esto se le suma la CTE "filtered_mentions" que permite filtrar y reducir el espacio de búsquedad antes de las agregaciones, minimizando el uso posterior de memoria.
* **tiempo de ejecución**: alto. Las operaciones de filtrado son caras pero se realizan al principio.
* **uso de CPU**: intermedio. Las operaciones de "unnest" es cpu expensive. Lo cuál puede aumentar los costos de billing en GCP.
* **uso de disco**: bajo. Idem que enfoque anterior. La extracción de usernames y counts se realizan en memoria, por lo que el acceso a disco es bajo (generalmente el sistema operativo o scheduler de la BD almacenando páginas).

Este enfoque es eficiente en memoria debido a: i) uso de "UNNEST" que descompone el array en elementos individuales, permitiendo procesar en siguientes pasos del query plan menos cantidad de data de forma progresiva, ii) el filtro inicial realizado por la CTE "filtered_mentiosn" que filtra y reduce el conjunto de datos antes de la agregación y iii) group y count se realiza después de haber reducido el total de datos.


#### Posible mejora (q3_memory):
* Indice en los campos de usuarios mencionados o materialized views para almacenar precomputacion de consultas frequentes. Ahora bien, el uso de CPU con indices se ve disminuido pero aumenta costo en disco (tradeoffs).

## Q3: SQL TIME

```python 
Q3_TIME_QUERY: str = r"""
WITH mention_counts AS (
  SELECT mentionedUser.username, COUNT(*) AS mention_count
  FROM `{file_path}`,
       UNNEST(mentionedUsers) AS mentionedUser
  WHERE mentionedUsers IS NOT NULL
    AND mentionedUser.username IS NOT NULL
  GROUP BY mentionedUser.username
)
SELECT username, mention_count
FROM mention_counts
ORDER BY mention_count DESC
LIMIT 10
"""
```

In [11]:
# q3_time 
import importlib
import q3_time
importlib.reload(q3_time)
result = q3_time.q3_time(file_path)
print(result)

2024-07-05 18:57:38,050 - INFO - Starting: q3_time
2024-07-05 18:57:38,051 - INFO - Ejecutando query
2024-07-05 18:57:39,561 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.51 segundos
2024-07-05 18:57:39,562 - INFO - Se uso query cache?: False
2024-07-05 18:57:39,563 - INFO - Bigquery Job Detail:
2024-07-05 18:57:39,564 - INFO - Job ID: aa238f0b-6963-4b96-a512-400b959ca8f5
2024-07-05 18:57:39,564 - INFO - Job Status: DONE
2024-07-05 18:57:39,565 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 22:57:38.762000+00:00
2024-07-05 18:57:39,566 - INFO - Tiempo fin en máquina GCP: 2024-07-05 22:57:39.147000+00:00
2024-07-05 18:57:39,566 - INFO - Tiempo de ejecución total en servidor GCP: 0.385 segundos
2024-07-05 18:57:39,566 - INFO - Tiempo de ejecución total en cliente (python): 1.51 segundos
2024-07-05 18:57:39,567 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.12 segundos
2024-07-05 18:57:39,568 - INFO - Bytes proce

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]


#### Pasos q3_time:

A diferencia del enfoque anterior, q3_time se enfoca en realizar la extracción, count y agregación en una sola ejecución/pasada. No utiliza múltiples etapas, disminuyendo la cantidad de steps en el query plan. No hay múltiples etapas de filtrado y agregación reduciendo el tiempo total de procesamiento. Mientras que q3_memory realiza un filtrado inicial antes de la agregación, introduciendo una etapa adicional de procesamiento (más tiempo de procesamiento pero menor uso de memoria).

Por otro lado, q3_time realiza la agrupación y count de forma inmediata en la misma CTE "mention_counts", optimizando el uso de CPU y latencia, que conlleva menor tiempo de procesamiento. Mientras que q3_memory primero filtra los datos en una CTE ("filtered_mentions") y luego realiza la agregación, castigando el tiempo de CPU pero disminuyendo el uso de memoria.

Por último, q3_time realiza todas las operaciones de filter, descomposición, agrupación y count en una sola ejecución/pasada, minimizando la cantidad de operaciones y reduciendo el tiempo de ejecución. Mientras que, q3_memory añade un paso adicional para filter y agrupar introduciendo más operaciones.


#### Análisis q3_time

* **uso de memoria**: intermedio. "unnest" explota los datos en este caso, pero no aumenta tanto debido a que se realiza en una sóla etapa de agregaciones y filtrados.
* **tiempo de ejecución**: bajo en comparación con enfoque anterior. Las operaciones se realizan en una sola ejecución, evitando pasos/instrucciones adicionales. Todo lo realiza en una sola CTE.
* **uso de CPU**: alto. Las operaciones de "unnest" es cpu expensive al igual que la agregación.
* **uso de disco**: bajo. Mismo argumento de enfoques pasados.


#### Posible mejora (q3_time):
* El formato de almacenamiento podría cambiar a Parquet o Avro para el almacenamiento, luego al ser comprimidos mejora el tiempo de lectura dependiendo de la consulta, por ejemplo, si es más columnar o row oriented.

## Tabla Comparativa de Mediciones Q3

### Configuración del job Q3

| Parámetros                                      | q3_memory | q3_time |
|------------------------------------------------|-----------|---------|
| QUERY_CACHE                                    | False     | False   |
| QUERY_PRIORITY                                 | INTERACTIVE     | INTERACTIVE|

### Resultados mediciones

| Medición                                      | q3_memory | q3_time |
|-----------------------------------------------|-----------|---------|
| Tiempo ejecución en cliente                   | 1.46 s    | 1.51 s  |
| Tiempo ejecución en GCP (server)              | 0.422 s   | 0.385 s |
| Delta tiempo (client-server)                  | 1.04 s    | 1.12 s  |
| Memoria usada por cliente python              | NA   | NA |
| CPU usado por cliente python                  | NA        | NA      |
| Bytes Procesados en GCP (server)              | 7.16 MB   | 7.16 MB |
| Bytes Facturados en GCP (server)              | 10.49 MB  | 10.49 MB|
| Slot Machine tiempo usado (GCP CPU Time) (ms) | 605 ms    | 562 ms  |
| BigQuery Total steps en QueryPlan             | 3         | 3       |

### Resultados Query Plan

#### Query Plan: q3_memory

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 31260           | COMPLETE |
| S01   | Sort+                | 31260        | 10              | COMPLETE |
| S02   | Output               | 10           | 10              | COMPLETE |

#### Query Plan: q3_time

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 31260           | COMPLETE |
| S01   | Sort+                | 31260        | 10              | COMPLETE |
| S02   | Output               | 10           | 10              | COMPLETE |

### Comparación Query Plan enfoque memory vs time

| Step | q3_memory (Q3_MEMORY_QUERY)                                   | q3_time (Q3_TIME_QUERY)                                    |
|------|---------------------------------------------------------------|-----------------------------------------------------------|
| S00  | **Input**: Records read: 117407, Records written: 31260, Status: COMPLETE | **Input**: Records read: 117407, Records written: 31260, Status: COMPLETE |
| S01  | **Sort+**: Records read: 31260, Records written: 10, Status: COMPLETE | **Sort+**: Records read: 31260, Records written: 10, Status: COMPLETE |
| S02  | **Output**: Records read: 10, Records written: 10, Status: COMPLETE | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |


Acá la diferencia entre un enfoque centrado en memory vs time es minima. 
Las razones de esto pueden ser:
1. La consulta q3_memory ya esta optimizada en terminos de uso de CPU y latencia. Por lo que no queda tanto margen de mejora.
2. OPORTUNIDAD DE MEJORA: los steps estan realizando las mismas operaciones de i/o por lo que la mejora no es explicado por uso de memoria y/o disco. Posiblemente la CPU o algún core paso menos tiempo (minimo) procesando alguna instrucción de las usadas en la consulta.
3. Otra explicación puede ser que la diferencia es minima para este dataset pero a medida que el dataset crezca se puede evidenciar una brecha más alta con respecto a la medición de tiempo de ejecución y slot machine (CPU time usado).
4. La única diferencia real es que el procesamiento bajo 40 ms con respecto a su contraparte q3_memory. No puedo atribuir que se deba a la optimización de la consulta sin realizar pruebas en profundidad y un análisis estadistico más pronfundo. Puede que simplemente sea el sistema operativo que mantuvo en cache o memoria ciertas páginas (pages) de rows de la BD como cache para acelerar futuras consultas (posible explicación es por el sistema operativo).